In [1]:
import pandas as pd


ds_json = pd.read_json('data/lorawan_antwerp_2019_dataset.json')
gw_loc = pd.read_json('data/lorawan_antwerp_gateway_locations.json')

In [2]:
# Loading initial position coordinates form machine learning predictions
pos_pred_rssi = pd.read_csv('files/position_pred_RSSI.csv', index_col=0)
pos_pred_comb = pd.read_csv('files/position_pred_weather-comb.csv', index_col=0)

import pymap3d as pm 

ref_pos = {'lat0': 51.260644,
           'lon0': 4.370656,
           'h0': 0}

# pos_pred_rssi['x'], pos_pred_rssi['y'], pos_pred_rssi['z'] = pm.geodetic2ecef(lat=pos_pred_rssi['lat'], lon=pos_pred_rssi['lon'], alt=0)
# pos_pred_rssi['x_i'], pos_pred_rssi['y_i'], pos_pred_rssi['z_i'] = pm.geodetic2ecef(lat=pos_pred_rssi['pred_lat_rssi'], lon=pos_pred_rssi['pred_lon_rssi'], alt=0)

# pos_pred_comb['x'], pos_pred_comb['y'], pos_pred_comb['z'] = pm.geodetic2ecef(lat=pos_pred_comb['lat'], lon=pos_pred_comb['lon'], alt=0)
# pos_pred_comb['x_i'], pos_pred_comb['y_i'], pos_pred_comb['z_i'] = pm.geodetic2ecef(lat=pos_pred_comb['pred_lat_comb'], lon=pos_pred_comb['pred_lon_comb'], alt=0)


pos_pred_rssi['x'], pos_pred_rssi['y'], pos_pred_rssi['z'] = pm.geodetic2enu(lat=pos_pred_rssi['lat'], lon=pos_pred_rssi['lon'], h=0, **ref_pos)
pos_pred_rssi['x_i'], pos_pred_rssi['y_i'], pos_pred_rssi['z_i'] = pm.geodetic2enu(lat=pos_pred_rssi['pred_lat_rssi'], lon=pos_pred_rssi['pred_lon_rssi'], h=0, **ref_pos)

pos_pred_comb['x'], pos_pred_comb['y'], pos_pred_comb['z'] = pm.geodetic2enu(lat=pos_pred_comb['lat'], lon=pos_pred_comb['lon'], h=0, **ref_pos)
pos_pred_comb['x_i'], pos_pred_comb['y_i'], pos_pred_comb['z_i'] = pm.geodetic2enu(lat=pos_pred_comb['pred_lat_comb'], lon=pos_pred_comb['pred_lon_comb'], h=0, **ref_pos)


In [3]:
pos_pred_rssi.to_csv('files/pos_pred_rssi_enu.csv')
pos_pred_comb.to_csv('files/pos_pred_comb_enu.csv')

In [4]:
import numpy as np
import pandas as pd
from itertools import combinations


# Collecting GW coordinates
def get_gw_cord_tdoa(index, ds_json, gw_loc):
    gw_ids = []
    toa = []
    gw_meta = ds_json.loc[index, ['gateways']]
    for gws in gw_meta:
        for gw in gws:
            gw_ids.append(gw['id'])
            toa.append(gw['rx_time']['time'])

    print(f'Receiving gateways: {gw_ids}')
    print(f'Time of arrivals: {toa}')
    
    gw_pos = []

    for i in gw_ids:
        gw = gw_loc[i]
        x, y, z = pm.geodetic2enu(lat=gw['latitude'], lon=gw['longitude'], h=0, **ref_pos)
        gw_pos.append([x,y,z])
    gw_positions = np.asarray(gw_pos)

    print('Gateway coordinates (enu): ')
    print(gw_positions)

    # Calulating TDoA value
    # We use the pandas timestamp method in this case, 
    # because it is the only one that can handle precision upto nano second  

    tdoa = []
    # Generate unique pairs using combinations
    for i, j in combinations(range(len(toa)), 2):
        diff = pd.Timestamp(toa[i]).value - pd.Timestamp(toa[j]).value
        diff_seconds = diff*1e-9
        tdoa.append([i, j, diff_seconds])
    print(f'Time difference of arrival: {tdoa}')

    return toa, gw_positions, tdoa


In [5]:
toa, gw_pos, tdoa = get_gw_cord_tdoa(127217, ds_json, gw_loc)

Receiving gateways: ['080E00B9', 'FF01072B', 'FF01753E']
Time of arrivals: ['2019-01-18T08:06:49.058+01:00', '2019-01-18T08:06:49.942733360+01:00', '2019-01-18T08:06:49.942734458+01:00']
Gateway coordinates (enu): 
[[ 3.42950136e+03 -7.28623463e+03 -5.08444638e+00]
 [ 3.92258394e+03 -7.27064010e+03 -5.35025029e+00]
 [ 2.51986817e+03 -7.02576446e+03 -4.36865519e+00]]
Time difference of arrival: [[0, 1, -0.88473336], [0, 2, -0.8847344580000001], [1, 2, -1.0980000000000001e-06]]


In [6]:
# Initital position for estimation 
row = pos_pred_comb[pos_pred_comb['gw_ref']==84169]
init_pos = [row['x_i'].values[0], row['y_i'].values[0]]

In [7]:
ts = np.asarray([pd.Timestamp(t) for t in toa])
measurements = np.c_[gw_pos[:, [0,2]], ts]

In [9]:
import numpy as np
import scipy.optimize as opt
import pymap3d as pm

# # Measurements (example with 3 measurements, can be extended to more)
# measurements = [
#     [34.888, -103.826, 0.0],
#     [34.931, -103.805, 0.6],
#     [34.921, -103.781, 2.5]
# ]


# Speed of propagation (m/s)
speed = 3e8

# Function to generate the residuals for all hyperbolae
def functions(measurements, speeds):
    def fn(args):
        x, y = args[:2]  # Extract x and y coordinates from args
        residuals = []
        for i in range(len(measurements)):
            xi, yi, ti = measurements[i]
            # We use the pandas timestamp method in this case, 
            # because it is the only one that can handle precision upto nano second
            diff_seconds = (pd.Timestamp(ti).value - pd.Timestamp(measurements[0][2]).value) * 1e-9  # the values are converted to seconds
            di = diff_seconds * speeds[i]
            ai = np.sqrt((x - xi)**2 + (y - yi)**2) - np.sqrt((x - measurements[0][0])**2 + (y - measurements[0][1])**2) - di
            residuals.append(ai)
        return residuals
    return fn

# Function to generate the Jacobian matrix
def jacobian(measurements, speeds):
    def fn(args):
        x, y = args[:2]  # Extract x and y coordinates from args
        jac = []
        for i in range(len(measurements)):
            xi, yi, ti = measurements[i]
            # We use the pandas timestamp method in this case, 
            # because it is the only one that can handle precision upto nano second
            diff_seconds = (pd.Timestamp(ti).value - pd.Timestamp(measurements[0][2]).value) * 1e-9  # the values are converted to seconds
            di = diff_seconds * speeds[i]
            adx = (x - xi) / np.sqrt((x - xi)**2 + (y - yi)**2) - (x - measurements[0][0]) / np.sqrt((x - measurements[0][0])**2 + (y - measurements[0][1])**2)
            ady = (y - yi) / np.sqrt((x - xi)**2 + (y - yi)**2) - (y - measurements[0][1]) / np.sqrt((x - measurements[0][0])**2 + (y - measurements[0][1])**2)
            jac.append([adx, ady])
        return np.array(jac)
    return fn

# # Initial estimate for x and y
# xp = np.mean([x for x, y, t in measurements])
# yp = np.mean([y for x, y, t in measurements])

# Extract measurements for use in functions and jacobian
speeds = [speed] * len(measurements)

# Define functions and jacobian
F = functions(measurements, speeds)
J = jacobian(measurements, speeds)

# Perform least squares optimization
# x, y, _ = opt.leastsq(F, x0=[xp, yp], Dfun=J)
x, y = opt.leastsq(func=F, x0=init_pos, Dfun=J)

print(f"Optimized (x, y): ({x}, {y})")


Optimized (x, y): ([252868.83010259   1930.09260213], 1)


In [10]:
init_pos

[1313.5715380541924, -6321.141998497997]

In [15]:
pm.enu2geodetic(e=x[0], n=x[1], u=0, **ref_pos)

(51.204706804272824, 7.988414419212078, 5000.4762988828425)

In [16]:
pm.enu2geodetic(e=init_pos[0], n=init_pos[1], u=0, **ref_pos)


(51.20382464462036, 4.389450921946975, 3.269191282091401)